In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None # None for auto detection
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.4.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [14]:
from datasets import load_dataset
import json

# Convert your JSONL to a format that can be loaded
# with open('hindi_chat.jsonl', 'r', encoding='utf-8') as f:
#     data = [json.loads(line) for line in f]

# # Save as a proper JSON file for loading
# with open('hindi_chat.json', 'w', encoding='utf-8') as f:
#     json.dump(data, f, ensure_ascii=False)

# Now load the dataset
dataset = load_dataset('json', data_files='hindi_chat.json')['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    # Convert each prompt-completion pair into a conversation format
    convos = []
    for prompt, completion in zip(examples["prompt"], examples["completion"]):
        convo = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": completion},
        ]
        convos.append(convo)

    # Apply the chat template
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
             for convo in convos]
    return {"text": texts}

# Apply the formatting
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,  # Adjust as needed
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

In [17]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 15 | Num Epochs = 30 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,5.922800
2,5.715000
3,5.086800
4,3.840200
5,2.902900
6,2.456000
7,1.765700
8,1.519800
9,1.154200
10,0.908400


In [20]:
FastLanguageModel.for_inference(model) # Enable faster inference

messages = [
    {"role": "user", "content": "tum kon hai"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 64,
    use_cache = True,
    temperature = 1.5,
    min_p = 0.1
)

print(tokenizer.batch_decode(outputs))

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntum kon hai<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nMain ek AI language model hoon, ConverseAI Labs ke liye fine-tuned.<|eot_id|>']


In [23]:
if True: model.push_to_hub_merged("raak-16/hinglish_model-ai", tokenizer, save_method = "merged_16bit", token = "your_token")


Unsloth: You are pushing to hub, but you passed your HF username = raak-16.
We shall truncate raak-16/hinglish_model-ai to hinglish_model-ai


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.84 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 54%|█████▎    | 15/28 [00:00<00:00, 22.41it/s]
We will save to Disk and not RAM now.
100%|██████████| 28/28 [00:40<00:00,  1.43s/it]


Unsloth: Saving tokenizer...

  0%|          | 0/1 [00:00<?, ?it/s]

...

README.md:   0%|          | 0.00/612 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/raak-16/hinglish_model-ai
